#**Recommandation de points d'intérêts sur les réseaux sociaux**

# 1. Introduction

L'objectif de ce TP est d'implémenter un algorithme de recommandation de points d'intérêt en se basant sur les données publiées sur les réseaux sociaux. Ces données couvrent en particulier les localisations des visites des utilisateurs ainsi que leurs relations sociales (ou "amis"), et peuvent être utilisées pour modéliser les préférences des utilisateurs et générer les recommandations. 

Ce document décrit les différentes étapes pour l'implémentation de l'approche *iGSLR* qui se base sur du filtrage collaboratif et sur l'exploitation des influences géographiques et sociales. 

*(Solution initialement présentée dans: "iGSLR: Personalized Geo-Social Location Recommendation: A Kernel Density Estimation Approach", Zhang and Chow, SIGSPATIAL'13)*

L'implémentation du modèle peut se faire à l'aide de la libraire **Surprise** (http://surpriselib.com/).


---


In [1]:
import pandas as pd
import os
import sys
import numpy as np
from geopy.distance import geodesic 
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import Dataset
import matplotlib.pyplot as plt
from surprise import AlgoBase
from surprise import PredictionImpossible
from itertools import combinations
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy

# 2. Chargement et traitement des données

Le jeu de données utilisé dans le cadre de ce TP a été collecté à partir du réseau social *Gowalla*. Il contient des informations concernant les profils des utilisateurs, leurs relations sociales et les localisations de leurs visites (qu'on associera à la dénomination "POI" dans la suite). 

Le jeu de données comprend 36,001,959 visites faites par 407,533 utilisateurs dans 2,724,891 POIs, et peut être téléchargé à partir de ce lien (https://drive.google.com/file/d/1DBq98KRWGRQyAJ0TqBqBKETGT90cyLwh/view?usp=sharing).

Dans la suite, il sera possible à tout moment de travailler sur un échantillon tiré de ce jeu de données, afin de faciliter les calculs. 

Afin de traiter et charger les données, effectuer les étapes suivantes:
*   Extraire le jeu de données et le charger dans un *dataframe* de *pandas*.



In [2]:
df_checkins = pd.read_csv("./gowalla/gowalla_checkins.csv")
df_friendship = pd.read_csv("./gowalla/gowalla_friendship.csv")
df_locations = pd.read_csv("./gowalla/gowalla_locations.csv")
df_userinfo = pd.read_csv("./gowalla/gowalla_userinfo.csv")

Select only a small dataset for comptung reasons

In [3]:
#df_checkins = df_checkins.head(200000)

*   Retirer les utilisateurs qui ont effectué moins de 5 visites.

In [4]:
# calculer le nombre de checkins pour chaque utilisateur
df_grouped_checkins  = df_checkins.groupby(['userid'] , as_index=False).count()

In [5]:
# recuperer seulement les uderid avec plus de 5 places visités et il faut réduire aussi à moins de 50 lieux à cause de la complexité
filtered_user_ids = df_grouped_checkins[(df_grouped_checkins.placeid >= 5) & (df_grouped_checkins.placeid <= 50)].userid.values 
df_filtered_checkins = df_checkins[df_checkins.userid.isin(filtered_user_ids)]

In [6]:
# Filtrer dtaframe df_friendship aussi
df_filtered_freindship = df_friendship[df_friendship.userid1.isin(filtered_user_ids)]

In [7]:
# Filtrer dtaframe userinfo aussi
df_filtered_userinfo = df_userinfo[df_userinfo.id.isin(filtered_user_ids)]

*   Associer à chaque utilisateur sa liste d'amis et placer le résultat dans un *dataframe*: *df_user_friends*.


In [8]:
df_user_friends = df_filtered_freindship.groupby('userid1').userid2.apply(list).reset_index(name='friends_list') 

* Calculer la fréquence de chaque paire *(utilisateur, POI)* et placer le résultat dans un *dataframe* *df_frequencies*.


In [9]:
# Joindre df_filtered_checkins avec df_locations en utilisant les champs place_id et id
df_checkins_locations = pd.merge(df_filtered_checkins, df_locations,left_on="placeid",right_on="id",how="left") 
df_checkins_locations = df_checkins_locations.dropna()

In [10]:
df_checkins_locations.head()

,userid,placeid,id,lng,lat,checkins_count,users_count
0,27506,332616,332616.0,-1.777557,52.411873,67.0,50.0
1,27506,32080,32080.0,-0.108469,51.555021,929.0,580.0
2,27506,1290090,1290090.0,-1.700092,52.189657,6.0,5.0
3,27506,1120128,1120128.0,-0.145848,51.513196,7.0,6.0
4,27506,445729,445729.0,-0.147148,51.513221,31.0,28.0


In [11]:
df_frequencies = df_checkins_locations.groupby(['userid', 'placeid'])["id"].count().reset_index(name="frequency")

In [12]:
# Joindre df_frequencies avec df_locations en utilisant les champs place_id et id
df_frequencies = pd.merge(df_frequencies, df_locations,left_on="placeid",right_on="id",how="left") 

* Mettre à jour les fréquences de *df_frequencies* pour les ramener à l'intervalle [0, 10] en appliquant une normalisation comme suit:

<center><img src='https://drive.google.com/uc?export=view&id=1vXvex4I5K30pmlsf2LQJ50nwCxt8XTA2' width=30%></center>

où $f_{min}$ et $f_{max}$ sont respectivement le nombre minimal et maximal de fréquences de visites d'un POI quelconque dans le jeu de données. 


In [13]:
# calculer f min
f_min = df_frequencies['frequency'].min()
# calculer f max
f_max = df_frequencies['frequency'].max()

In [14]:
df_frequencies["ratings"] = df_frequencies["frequency"].apply(lambda x: 10*np.tanh(10*(x-f_min)/(f_max-f_min)))

In [15]:
df_frequencies.head()

,userid,placeid,frequency,id,lng,lat,checkins_count,users_count,ratings
0,15,8904,1,8904,-94.607499,39.052318,114,21,0.000000
1,15,8947,2,8947,-122.029631,37.331880,3100,1186,2.186351
2,15,9073,1,9073,-122.393725,37.795339,2654,1659,0.000000
3,15,9186,1,9186,-77.036594,38.897638,1603,1383,0.000000
4,15,9591,1,9591,-122.159772,37.447908,1418,525,0.000000


* Charger *df_frequencies* dans le framework *Suprise* en utilisant la fonction *load_from_df()*.


In [16]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_frequencies[['userid', 'placeid', 'ratings']], reader)

* Utiliser la fonction *train_test_split()* pour diviser *df_frequencies* en un ensemble d'apprentissage (*training set*, 75\% du jeu de données) et un ensemble de test (*test set*, 25\% du jeu de données). 


In [17]:
trainset, testset = train_test_split(data, test_size=.25)

*  Associer à chaque utilisateur sa liste de POI visités et placer le résultat dans un *dataframe* *df_user_POI*.



---

In [18]:
df_user_POI = df_frequencies.groupby('userid').placeid.apply(list).reset_index(name='POI_list') 

In [19]:
df_user_POI.head()

,userid,POI_list
0,15,"[8904, 8947, 9073, 9186, 9591, 10299, 14710, 2..."
1,37,"[10817, 34906, 39911, 43229, 44243, 59855, 628..."
2,45,"[22370, 32398, 34157, 37303, 6563736, 7360178]"
3,53,"[17579, 286512, 290022, 644423, 1541920]"
4,81,"[9220, 9221, 9222, 9226, 9246, 9247, 9248, 924..."


# 3. Influence géographique

Le *dataframe* *df_user_POI* associe chaque utilisateur $u$ à la liste $L_u$ des POIs qu'il a visité. 


*   Utiliser *df_user_POI* pour calculer pour chaque utilisateur $u$ les distances $d_{ij}$ entre chaque paire de POIs visités: 

$\forall p_i, p_j \in L_u \times L_u, d_{ij} = distance(p_i, p_j)$. 

On dénotera cette liste de distances par $D_u$.



In [20]:
def distance(pi, pj):
    
    # recuperer latitute et longitude de premier place
    lat0 = df_locations[df_locations.id == pi].lat.values
    lng0 = df_locations[df_locations.id == pi].lng.values

    # recuperer latitute et longitude de second place
    lat1 = df_locations[df_locations.id == pj].lat.values
    lng1 = df_locations[df_locations.id == pj].lng.values

    # calculer distance in km en utilisant la librairie geopy

    return geodesic((lat0,lng0), (lat1,lng1)).km

def distance_pair_list(Lu):
    # recuper toutes les combinaisons de paires de lieux deja visités
    if len(Lu) != 1 :
        pairs = list(combinations(Lu,2))
    else:
        pairs = []
    
    dist_list = []
    
    #calucler la distance entre chaque paire de place
    if pairs != [] :
        for pair in pairs :
            dist_list.append(distance(pair[0], pair[1]))
    return dist_list

* Utiliser $D_u$ pour calculer la densité $f_u$ d'une distance quelconque comme suit:

<center><img src='https://drive.google.com/uc?export=view&id=1BXoT3vKbXQ6GllTIQkUIXB12xh72XiN5' width=30%/></center>

$K(.)$ est choisi comme étant la densité d'une fonction gaussienne:

<center><img src='https://drive.google.com/uc?export=view&id=17QL_BH-mnu0IUAgJwGv-G7GUjNJF30Ac' width = 18%></center>

Le paramètre de lissage $h = 1,06\hat{\sigma}n^{-1/5}$, où $n$ est le nombre de POIs présents dans $L_u$ et $\hat{\sigma}$ est l'écart type de $D_u$. Implémenter l'expression $\hat{f}_u$ dans une fonction *density()*.

In [21]:
def k_gaussian(x):
    return (1/np.sqrt(2*np.pi))*np.exp(-x**2/2)

def density(Du, dij, h):
    D2 = list(map(lambda x : (dij-x)/h,Du))
    density = 1/(len(Du)*h)*sum(list(map(k_gaussian,D2)))
    return density

* La densité $\hat{f}_u$ est utilisée pour estimer la probabilité qu'un POI $p_i$ non visité par un utilisateur $u$ corresponde aux préférences géographiques de $u$ étant donné son historique de visite. Afin d'obtenir cette probabilité, on calcule la distance entre $p_i$ et chacun des POIs de la liste $L_u$ et on estime ensuite la probabilité de chacune de ces distances en passant par $\hat{f}_u$.

* La probabilité finale qu'un utilisateur $u$ visite un POI $p_i$ est obtenue comme suit:

<center><img src='https://drive.google.com/uc?export=view&id=1jq6OspCkQegNTWm6yDb8VEQFtrLOpDr_' width=30%/> </center>

**P**(p_i \| L_u) représente la probabilité que l'utilisateur $u$ visite le POI $p_i$ étant donné le critère géographique. Implémenter l'équation ci-dessus dans une fonction *geo_proba()* qui prend en entrée la liste $L_u$ et un POI, et qui retourne la probabilité de visite de ce POI. 

In [22]:
def geo_proba(Lu, pi):

    d = []
    
    # recuperer la longitude et la latitude de lieu candidat à recommender
    lat_i =   df_locations[df_locations.id == pi].lat.values[0]
    long_i =  df_locations[df_locations.id == pi].lng.values[0]
    
    # calculer toutes les distances entre le lieu candidat à recommender et chaque lieu déja visité dans liste Lu
    for l in Lu :
        long_j = df_locations[df_locations.id == l].lng.values[0]
        lat_j = df_locations[df_locations.id == l].lat.values[0]
        d.append(geodesic((lat_j,long_j), (lat_i,long_i)).km)
    
    # recuperer la distances entre chaque paire des lieux deja visité
    Du = distance_pair_list(Lu)
    
    # calculer h
    n = len(Lu)
    sigma = np.std(Du)
    h = 1.06*sigma*n**(-1/5)
    
    # calucler la densité
    density_list = list(map(lambda x : density(Du, x, h),d))

    return np.mean(density_list)



---



# 4. Influence sociale

Le *dataframe* *df_user_friends* associe chaque utilisateur $u$ à ses amis $F(u)$.

*   Pour chaque paire d'utilisateurs $(u, v)$, on calcule leur *similarité sociale* en utilisant le coefficient de Jaccard comme suit:

<center><img src='https://drive.google.com/uc?export=view&id=1hwScp-pmMvNgNxduEdSUkIs4f7U9GEvE' width=30%></center>

Implémenter ce coefficient dans la fonction *social_similarity()*.


In [23]:
def social_similarity(u, v):
    # amis de u
    list1 = df_user_friends[df_user_friends.userid1 == u].friends_list.values[0]
    # amis de v
    list2 = df_user_friends[df_user_friends.userid1 == v].friends_list.values[0]
    sim = len(list(set(list1) & set(list2))) / len(list(set(list1) | set(list2)))

* Ce coefficient peut être utilisé dans un modèle de filtrage collaboratif comme suit:

<center><img src='https://drive.google.com/uc?export=view&id=1wGcIMVrXYiBOeOr7W35hrJdrisErcBxd' width=30%></center>

$r_{ui}$ indique la fréquence de visite de $u$ dans $i$, extraite de *df_frequencies*.

In [24]:
def get_rate(u,i,trainset) : 
        
    inner_iid = trainset.to_inner_iid(i) # get the inner id of the location
    inner_uid = trainset.to_inner_uid(u) # get the inner id of the user

    res_ir = trainset.ir[inner_iid] # get rates given for the location
    uid_ir = list(map(lambda x :x[0],res_ir)) 
    rate_ir = list(map(lambda x :x[1],res_ir)) 

    if uid_ir.count(inner_uid) == 1 :
        r = rate_ir[uid_ir.index(inner_uid)]
    
    # si le lieu n'a pas été noté par u
    else : 
        r = 0
    return r


In [25]:
def r_hat(user_i,j,trainset,F_i) :

    rate_j = np.zeros(len(F_i))
    sim_list = np.zeros(len(F_i))

    for i,user in enumerate(F_i) :

        rate_j[i] = get_rate(user,j,trainset)
        sim_list[i] = social_similarity(user_i,user)

    return np.dot(sim_list,rate_j) / max(np.sum(sim_list),0.01)

* La prédiction $\hat{r}_{ui}$ peut être tranformée en probabilité comme suit:

<center><img src='https://drive.google.com/uc?export=view&id=1v1b0LZrsAdT8bBi-zrJjRjUnhAiBF-SY' width=30%></center>

où $L$ est la liste de POIs. 

In [26]:
def user_is_in_trainset(u) :

    try :
        trainset.to_inner_uid(u)
        res = 1
    except :
        res = 0

    return res

In [27]:
def item_is_in_trainset(i) :

    try :
        trainset.to_inner_iid(i)
        res = 1
    except :
        res = 0

    return res

In [28]:
def social_proba(u, i, Lu):
    #reuperer la list de tous les lieux
    L = list(np.unique(df_locations.id))
    #recuperer la liste des lieux visités par u
    # recuperer la liste des lieux qui n'ont pas été encore visités par user u
    L_diff = list(set(L)- set(Lu))
    # considerer seulement les lieux qui sont dans le training set
    L_diff = [i for i in L_diff if item_is_in_trainset(i)]
    # reduire la liste de lieux à recommender à 50 pour raison de complexité de calcul
    L_diff = L_diff[:200]
    # recuperer la liste des amis de u
    F_i = df_user_friends[df_user_friends.userid1 == u].friends_list.values[0]
    # consider seulement la liste des amis qui sont dans le training set
    print()
    F_i = [i for i in F_i if user_is_in_trainset(i)]

    if F_i == [] :

        return np.nan

    else :

        numerator = r_hat(u,i,trainset,F_i)
        denominator = max(max(list(map(lambda x : r_hat(u,x,trainset,F_i),L_diff))),0.01)
        
        return numerator/denominator



---



# 5. Génération et évaluation des recommandations

* Afin de générer les recommandations, on calcule les scores de pertinence $\hat{s}_{ui}$ pour un utilisateur $u$ et un POI $i$ comme suit:

<center> <img src='https://drive.google.com/uc?export=view&id=1Z2GsqBKynyHXE0or99SB2PK3dRTsyIsg' width=20%> </center>

* L'équation ci-dessus doit être implémenter dans une nouvelle classe de recommandation. (https://surprise.readthedocs.io/en/stable/building_custom_algo.html)

In [29]:
class GSLR(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)

  
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        self.trainset = trainset
        return self

    def estimate(self, u, i): 
        
        # Liste des lieux visité par utilisateur u
        Lu = list(set(df_user_POI[df_user_POI.userid == u].POI_list.values[0]))
        
        
        if not user_is_in_trainset(u) or not item_is_in_trainset(i):
            return np.nan

        else :       
            if np.isnan(social_proba(u, i, Lu)):
                score = geo_proba(Lu, i)
            else :
                score = (geo_proba(Lu, i) + social_proba(u, i, Lu)) / 2

        return score

In [30]:
gslr = GSLR()
gslr.fit(trainset)
predictions = []

for i in range(len(testset[:10])):
    prediction = gslr.estimate(testset[i][0], testset[i][1])
    if np.isnan(prediction):
        print('Utilisateur ', testset[i][0], " et/ou lieu ", testset[i][1], " non existants dans la base.")
    else:
        print("Probability of user ", testset[i][0], " visits ", testset[i][1], " is ", prediction)
    predictions.append(prediction)


Utilisateur  121599  et/ou lieu  420048  non existants dans la base.
Utilisateur  261561  et/ou lieu  17646  non existants dans la base.
Utilisateur  380243  et/ou lieu  619514  non existants dans la base.

Probability of user  2634921  visits  6758198  is  0.02567067743028482
Utilisateur  391630  et/ou lieu  554889  non existants dans la base.
Utilisateur  351034  et/ou lieu  2962070  non existants dans la base.

Probability of user  2163956  visits  6600101  is  0.0007908749364762526

Probability of user  828748  visits  45307  is  0.05811903112962183
Utilisateur  70204  et/ou lieu  169342  non existants dans la base.

Probability of user  2207714  visits  335738  is  0.0005633826511897169


* Générer les recommandations pour chaque utilisateur inclus dans l'ensemble de test et calculer la RMSE, la précision (Precision@N) et le recall (Recall@N) de l'approche (plus de détails concernant la validation croisée ici: https://surprise.readthedocs.io/en/stable/model_selection.html). 

* Comparer la performance de ce modèle avec les modèles standards implémentés dans *Surprise*.

In [ ]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), NMF()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=2, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')   